In [7]:
import talib
import numpy as np
import requests
import pandas as pd
import time
import datetime
from ciso8601 import parse_datetime

from ftx_client import FtxClient

### Notes

- Currently, the price is the last traded price. We may want to change it to the average price in the last window

### Api keys and inits

In [19]:
# api_key_test = 'wiYFjNZa8vQZ00HVlvviBZU8KlUk3P9BsOaFNgWQ'
# api_secret_test = 'FV3I-fMloT9dVIO5W1twDUZA385Q06VDjrN2mqeV'
# api_key = '4MZCayt-zZKp3QQFOBgPFdwOK5B1zoxJPMeFAGKz'
# api_secret = 'cfQxfctS5j7lR0iq-iGsiItt4znrH9P4AkF-IzMH'

with open('api_keys.txt', 'r') as f:
    lines = [x.rstrip('\n') for x in f.readlines()]
    api_key = lines[0]
    api_secret = lines[1]

curr_pairs = ['BTC', 'USD']
curr_symbol = '{}/{}'.format(curr_pairs[0], curr_pairs[1])
rsi_resolution = 900 # 15 minutes in seconds

logfile = 'ftx_log.log'
with open(logfile, 'w') as f:
    pass

client = FtxClient(api_key=api_key, api_secret=api_secret)

### Utility Functions

In [16]:
def logTrade(oldBalance, newBalance, branch, enterTime, enterPrice, exitPrice, orderid):
    with open(logfile, 'a') as f:
        f.write(f'oldBalance: {oldBalance}, newBalance: {newBalance}, branch: {branch}, enterTime: {enterTime}, enterPrice: {enterPrice}, exitPrice: {exitPrice}, orderid: {orderid}\n')
        
def Stoch(close,high,low, smoothk, smoothd, n):
    lowestlow = pd.Series.rolling(low,window=n,center=False).min()
    highesthigh = pd.Series.rolling(high, window=n, center=False).max()
    K = pd.Series.rolling(100*((close-lowestlow)/(highesthigh-lowestlow)), window=smoothk).mean()
    D = pd.Series.rolling(K, window=smoothd).mean()

    return K, D

def get_RSI_df():
#     candles = client.get_klines(symbol=curr_symbol, interval=Client.KLINE_INTERVAL_15MINUTE)
    candles = client.get_candles(curr_symbol, rsi_resolution)
    opens = [x['open'] for x in candles]
    closes = [x['close'] for x in candles]
    highs = [x['high'] for x in candles]
    lows = [x['low'] for x in candles]
    startTimes = [x['time'] for x in candles]
    volumes = [x['volume'] for x in candles]
    df = pd.DataFrame(list(zip(opens, closes, highs, lows, startTimes, volumes)), columns=['open', 'close', 'high', 'low', 'startTime', 'volume'])
#     df.columns=['timestart','open','high','low','close','?','timeend','?','?','?','?','?']
#     df.timestart = [datetime.datetime.fromtimestamp(i/1000) for i in df.timestart.values]
#     df.timeend = [datetime.datetime.fromtimestamp(i/1000) for i in df.timeend.values]
    
    float_data = [float(x) for x in df.close.values]
    np_float_data = np.array(float_data)
    rsi = talib.RSI(np_float_data, 14)
    df['rsi'] = rsi
#     print('rsi: ', df['rsi'].astype(float).iloc[-1])

    stochrsi = Stoch(df.rsi, df.rsi, df.rsi, 3, 3, 14)
    df['StochrsiK'],df['StochrsiD'] = stochrsi
#     print('stochrsiK: ', df['StochrsiK'])
    
    return df

def get_RSI():
    df = get_RSI_df()
    newestK = df.StochrsiK.astype(float).iloc[-1]
    return newestK

def print_log(s):
    with open(logfile, 'a') as f:
        f.write(s + '\n')

### Testing (Check balance)

In [17]:
price = client.get_price(curr_symbol)
print(price)
print(client.get_account_info())
print()
print('hi, ', client.get_balance(curr_pairs[1]))
print('bye, ', client.get_balances())
print()
candles = client.get_candles(curr_symbol, rsi_resolution)
print(len(candles))
print(candles[3])
print(parse_datetime(candles[3]['startTime']).timestamp())
print(get_RSI())
print('#############')
print()
# print(client.get_order_status(7204332489))

11814.0
{'backstopProvider': False, 'chargeInterestOnNegativeUsd': False, 'collateral': 2649.30346873, 'freeCollateral': 2649.30346873, 'initialMarginRequirement': 0.1, 'leverage': 10.0, 'liquidating': False, 'maintenanceMarginRequirement': 0.03, 'makerFee': 0.0002, 'marginFraction': None, 'openMarginFraction': None, 'positionLimit': None, 'positionLimitUsed': None, 'positions': [], 'spotLendingEnabled': False, 'spotMarginEnabled': False, 'takerFee': 0.0007, 'totalAccountValue': 2649.30346873, 'totalPositionSize': 0.0, 'useFttCollateral': True, 'username': 'quant@sminem.com'}

hi,  2649.30346873
bye,  [{'coin': 'USD', 'free': 2649.30346873, 'total': 2649.30346873, 'usdValue': 2649.30346873}]

1500
{'close': 9329.0, 'high': 9331.0, 'low': 9325.0, 'open': 9330.5, 'startTime': '2020-07-22T10:30:00+00:00', 'time': 1595413800000.0, 'volume': 6694.1964}
1595413800.0
78.20336737828804
#############



### Testing (Place/cancel order)

In [5]:
# Place order
print(price)
# enterLong(price)
# exitLong()
# client.cancel_order(7226227238)

11294.0


### Testing (Check open orders, order history)

In [18]:
# Check order status
def print_side_orders(side):
    orders = client.get_order_history(market=curr_symbol)
    for order in orders:
        if order['side'] == side:
            print(order)

orders = client.get_open_orders(market=curr_symbol)
print('open orders: ', orders)
history = client.get_order_history(market=curr_symbol)
print(len(history))
print(history)
print()
# print(history[0])
# print(history[1])
print()

print_side_orders('buy')
print()
print_side_orders('sell')

# exitLong(price)

open orders:  []
0
[]





### Trading Functions

In [13]:
def enterLong(price):
    curr_usd = client.get_balance('USD')
    usd_per_btc = client.get_price(curr_symbol)
    btc = curr_usd / usd_per_btc
    newprice = price + 1
    
    # Market order
    result = client.place_order(market=curr_symbol, side='buy', price=price, size=btc, type='limit')
    print('res: ', result)
    orderid = result['id']
    
    # Wait for market order to get filled
    while client.get_order_status(orderid)['status'] != 'closed':
        time.sleep(3)
        
    return orderid
    
def exitLong():
    curr_btc = client.get_balance('BTC')
#     print('curr_btc: ', curr_btc)
    curr_usdvalue = client.get_usdValue('BTC')
    price = curr_usdvalue / curr_btc
    newprice = price - 5
    
    result = client.place_order(market=curr_symbol, side='sell', price=newprice, size=curr_btc, type='limit', ioc=False)
    print('res: ', result)
    
    orderid = result['id']
    
    # Wait for market order to get filled
    while client.get_order_status(orderid)['status'] != 'closed':
        time.sleep(3)
        
    return orderid

### Initialize variables (balance, orderDetails, activeLong, etc)

In [20]:
gap = 100.0
rsi_low_thresh = 33.0

activeLong = False
enterPrice = None
exitPrice = None
enterTime = None

### Main Loop

In [9]:
while True:
    time.sleep(5)
    
    currPrice = client.get_price(curr_symbol)
    nearestMult = (currPrice//gap) * gap
    modPrice = currPrice - nearestMult
    centPrice = currPrice // gap
    
    # Get RSI
    rsi = get_RSI()
    print('stochRSI: ', rsi)
#     print(type(newestK))
    
    if not activeLong and rsi < rsi_low_thresh and modPrice <= 0.2*gap:
        oldBalance = client.get_balance('USD')
        orderid = enterLong(currPrice)
        print('Entering long with {} USD at {}'.format(oldBalance, currPrice))
        activeLong = True
        enterPrice = currPrice + 1
        enterTime = int(time.time())
        print_log('Entering long with {} USD at {} at {} orderid = {}'.format(oldBalance, currPrice, enterTime, orderid))
        
    if activeLong and enterPrice < currPrice and modPrice >= 0.8*gap:
        orderid = exitLong()
        print('Exiting long at {}'.format(currPrice))
        logTrade(oldBalance, newBalance=client.get_balance('USD'), branch='Success', enterTime=enterTime, enterPrice=enterPrice, exitPrice=currPrice, orderid=orderid)
        activeLong = False
        
    if activeLong and enterPrice - currPrice > 0.5*gap:
        orderid = exitLong()
        exitPrice = currPrice - 5
        print('Exiting long at {}'.format(exitPrice))
        activeLong = False
        logTrade(oldBalance, newBalance=client.get_balance('USD'), branch='Cut loss', enterTime=enterTime, enterPrice=enterPrice, exitPrice=currPrice, orderid=orderid)

stochRSI:  28.74606775298457
stochRSI:  28.74606775298457
stochRSI:  28.74606775298457
stochRSI:  30.46885090026278
stochRSI:  30.46885090026278
stochRSI:  30.46885090026278
stochRSI:  30.46885090026278
stochRSI:  30.46885090026278
stochRSI:  30.46885090026278
stochRSI:  30.46885090026278
stochRSI:  30.46885090026278
stochRSI:  30.46885090026278
stochRSI:  30.46885090026278
stochRSI:  30.46885090026278
stochRSI:  30.46885090026278
stochRSI:  30.46885090026278
stochRSI:  30.46885090026278
stochRSI:  30.46885090026278
stochRSI:  30.46885090026278
stochRSI:  30.46885090026278
stochRSI:  30.46885090026278
stochRSI:  30.46885090026278
stochRSI:  30.46885090026278
stochRSI:  30.46885090026278
stochRSI:  30.46885090026278
stochRSI:  29.5356076101272
stochRSI:  29.5356076101272
stochRSI:  29.5356076101272
stochRSI:  29.5356076101272
stochRSI:  29.5356076101272
stochRSI:  29.5356076101272
stochRSI:  29.5356076101272
stochRSI:  29.5356076101272
stochRSI:  29.5356076101272
stochRSI:  29.535607610

stochRSI:  18.46069555926184
stochRSI:  18.46069555926184
stochRSI:  18.46069555926184
stochRSI:  18.46069555926184
stochRSI:  18.46069555926184
stochRSI:  18.46069555926184
stochRSI:  18.46069555926184
stochRSI:  18.46069555926184
stochRSI:  18.46069555926184
stochRSI:  18.46069555926184
stochRSI:  18.46069555926184
stochRSI:  18.46069555926184
stochRSI:  18.46069555926184
stochRSI:  18.46069555926184
stochRSI:  14.147114919383966
stochRSI:  14.147114919383966
stochRSI:  14.147114919383966
stochRSI:  14.147114919383966
stochRSI:  14.147114919383966
stochRSI:  14.147114919383966
stochRSI:  14.147114919383966
stochRSI:  14.147114919383966
stochRSI:  14.147114919383966
stochRSI:  14.147114919383966
stochRSI:  11.183152599159575
res:  {'avgFillPrice': None, 'clientId': None, 'createdAt': '2020-08-04T03:31:54.097979+00:00', 'filledSize': 0.0, 'future': None, 'id': 7241440428, 'ioc': False, 'liquidation': False, 'market': 'BTC/USD', 'postOnly': False, 'price': 11252.0, 'reduceOnly': False, 

stochRSI:  0.0
stochRSI:  0.0
stochRSI:  0.0
stochRSI:  0.0
stochRSI:  0.0
stochRSI:  0.0
stochRSI:  0.0
stochRSI:  0.0
stochRSI:  0.0
stochRSI:  0.0
stochRSI:  0.0
stochRSI:  0.0
stochRSI:  0.0
stochRSI:  0.0
stochRSI:  0.0
stochRSI:  0.0
stochRSI:  0.0
stochRSI:  0.0
stochRSI:  0.0
stochRSI:  1.1165562127477315
stochRSI:  1.1165562127477315
stochRSI:  1.1165562127477315
stochRSI:  1.1165562127477315
stochRSI:  1.1165562127477315
stochRSI:  1.1165562127477315
stochRSI:  1.1165562127477315
stochRSI:  1.1165562127477315
stochRSI:  1.1165562127477315
stochRSI:  1.1165562127477315
stochRSI:  1.1165562127477315
stochRSI:  1.300712794451935
stochRSI:  1.300712794451935
stochRSI:  1.300712794451935
stochRSI:  1.300712794451935
stochRSI:  1.300712794451935
stochRSI:  1.300712794451935
stochRSI:  1.300712794451935
stochRSI:  1.300712794451935
stochRSI:  1.300712794451935
stochRSI:  1.300712794451935
stochRSI:  1.300712794451935
stochRSI:  4.037436285295024
stochRSI:  4.037436285295024
stochRSI

stochRSI:  22.360806793107688
stochRSI:  19.46145260063402
stochRSI:  19.46145260063402
stochRSI:  19.46145260063402
stochRSI:  19.46145260063402
stochRSI:  19.46145260063402
stochRSI:  19.46145260063402
stochRSI:  19.46145260063402
stochRSI:  19.46145260063402
stochRSI:  19.46145260063402
stochRSI:  19.46145260063402
stochRSI:  19.46145260063402
stochRSI:  21.581377332774505
stochRSI:  21.581377332774505
stochRSI:  21.581377332774505
stochRSI:  21.581377332774505
stochRSI:  21.581377332774505
stochRSI:  21.581377332774505
stochRSI:  21.581377332774505
stochRSI:  21.581377332774505
stochRSI:  21.581377332774505
stochRSI:  21.581377332774505
stochRSI:  21.581377332774505
stochRSI:  22.51814574542911
stochRSI:  22.51814574542911
stochRSI:  22.51814574542911
stochRSI:  22.51814574542911
stochRSI:  22.51814574542911
stochRSI:  22.51814574542911
stochRSI:  22.51814574542911
stochRSI:  22.51814574542911
stochRSI:  22.51814574542911
stochRSI:  22.51814574542911
stochRSI:  22.51814574542911
st

Entering long with 145.33858689 USD at 11303.0
stochRSI:  30.88183334259698
stochRSI:  30.88183334259698
stochRSI:  34.12354416224775
stochRSI:  34.12354416224775
stochRSI:  34.12354416224775
stochRSI:  34.12354416224775
stochRSI:  34.12354416224775
stochRSI:  34.12354416224775
stochRSI:  34.12354416224775
stochRSI:  34.12354416224775
stochRSI:  34.12354416224775
stochRSI:  34.12354416224775
stochRSI:  34.12354416224775
stochRSI:  35.846315695256
stochRSI:  35.846315695256
stochRSI:  35.846315695256
stochRSI:  35.846315695256
stochRSI:  35.846315695256
stochRSI:  35.846315695256
stochRSI:  35.846315695256
stochRSI:  35.846315695256
stochRSI:  35.846315695256
stochRSI:  35.846315695256
stochRSI:  35.846315695256
stochRSI:  35.846315695256
stochRSI:  35.67644608416661
stochRSI:  35.67644608416661
stochRSI:  35.67644608416661
stochRSI:  35.67644608416661
stochRSI:  35.67644608416661
stochRSI:  35.67644608416661
stochRSI:  35.67644608416661
stochRSI:  35.67644608416661
stochRSI:  35.676446

stochRSI:  63.299643316213455
stochRSI:  63.299643316213455
stochRSI:  63.299643316213455
stochRSI:  63.299643316213455
stochRSI:  63.299643316213455
stochRSI:  63.299643316213455
stochRSI:  63.299643316213455
stochRSI:  63.299643316213455
stochRSI:  63.299643316213455
stochRSI:  63.299643316213455
stochRSI:  63.299643316213455
stochRSI:  57.560614249473076
stochRSI:  57.560614249473076
stochRSI:  57.560614249473076
stochRSI:  57.560614249473076
stochRSI:  57.560614249473076
stochRSI:  57.560614249473076
stochRSI:  57.560614249473076
stochRSI:  57.560614249473076
stochRSI:  57.560614249473076
stochRSI:  57.560614249473076
stochRSI:  57.560614249473076
stochRSI:  57.0164248199456
stochRSI:  57.0164248199456
stochRSI:  57.0164248199456
stochRSI:  57.0164248199456
stochRSI:  57.0164248199456
stochRSI:  57.0164248199456
stochRSI:  57.0164248199456
stochRSI:  57.0164248199456
stochRSI:  57.0164248199456
stochRSI:  57.0164248199456
stochRSI:  57.0164248199456
stochRSI:  58.4791252019706
stoc

ConnectionError: HTTPSConnectionPool(host='ftx.com', port=443): Max retries exceeded with url: /api/markets/BTC/USD (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f039b268da0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution',))